In [1]:
#import transformers
import os
import re
import string
import pandas as pd
#import torch
#from huggingface_hub import notebook_login
#from transformers import pipeline
#from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModel

In [3]:
# nltk
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stoplist=stopwords.words('english')
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer= WordNetLemmatizer()
from nltk.corpus import wordnet

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')

# Importing gensim related libraries
import gensim
from gensim.models.ldamulticore import LdaMulticore
from gensim.corpora import Dictionary
from gensim.models import Phrases
from collections import Counter
from gensim.models import Word2Vec

[nltk_data] Downloading package stopwords to /mnt/ceph/storage/data-
[nltk_data]     tmp/teaching-current//aq60qovu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /mnt/ceph/storage/data-
[nltk_data]     tmp/teaching-current//aq60qovu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /mnt/ceph/storage/data-tmp/teaching-
[nltk_data]     current//aq60qovu/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /mnt/ceph/storage/data-
[nltk_data]     tmp/teaching-current//aq60qovu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /mnt/ceph/storage/data-
[nltk_data]     tmp/teaching-current//aq60qovu/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [4]:
df_orig = pd.read_csv('../../datasets/future_statements_dataset/future_dataset.csv', sep='|')

In [5]:
df_orig.columns

Index(['statement', 'future', 'source'], dtype='object')

In [6]:
def get_pos_tag(tag):
    """This function is used to get the part-of-speech(POS) for lemmatization"""
    if tag.startswith('N') or tag.startswith('J'):
        return wordnet.NOUN
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN #default case

In [7]:
def preprocess(text):
    """ 1. Removes Punctuations
        2. Removes words smaller than 3 letters
        3. Converts into lowercase
        4. Lemmatizes words
        5. Removes Stopwords
    """   
    punctuation= list(string.punctuation)
    doc_tokens= nltk.word_tokenize(text)
    word_tokens= [word.lower() for word in doc_tokens if not (word in punctuation or len(word)<=3)]
    
    # Lemmatize    
    pos_tags=nltk.pos_tag(word_tokens)
#     print(pos_tags)
    doc_words=[wordnet_lemmatizer.lemmatize(word, pos=get_pos_tag(tag)) for word, tag in pos_tags]
    doc_words= [word for word in doc_words if word not in stoplist]
    
    return doc_words

df_clean = df_orig['statement'].apply(preprocess)
df_clean.head()

0    [need, rededicate, principle, purpose, organiz...
1     [many, thing, leave, behind, many, thing, await]
2    [philippine, stand, ready, assist, implement, ...
3    [first, indigenous, population, paraguay, larg...
4    [also, grateful, principled, position, home, m...
Name: statement, dtype: object

In [8]:
docs= list(df_clean)
phrases = gensim.models.Phrases(docs, min_count=10, threshold=20)
bigram_model = gensim.models.phrases.Phraser(phrases)

In [9]:
def make_bigrams(texts):
    return [bigram_model[doc] for doc in texts]

# Form Bigrams
data_words_bigrams = make_bigrams(docs)

In [22]:
# Checkout most frequent bigrams :
bigram_counter1= Counter()
for key in phrases.vocab.keys():
    if key not in stopwords.words('english'):
        if len(str(key).split('_'))>1:
            bigram_counter1[key]+=phrases.vocab[key]

for key, counts in bigram_counter1.most_common(10):
    print(key,">>>>", counts)

united_nation >>>> 100
international_community >>>> 38
security_council >>>> 31
general_assembly >>>> 29
unite_nation >>>> 24
develop_country >>>> 22
next_year >>>> 20
climate_change >>>> 19
last_year >>>> 19
peace_security >>>> 17


In [20]:
w2vmodel.wv.key_to_index

{'world': 0,
 'year': 1,
 'people': 2,
 'go': 3,
 'country': 4,
 'make': 5,
 'time': 6,
 'international': 7,
 'technology': 8,
 'state': 9,
 'united_nation': 10,
 'government': 11,
 'take': 12,
 'many': 13,
 'company': 14,
 'future': 15,
 'need': 16,
 'work': 17,
 'become': 18,
 'first': 19,
 'come': 20,
 'also': 21,
 'support': 22,
 'problem': 23,
 'development': 24,
 'change': 25,
 'continue': 26,
 'data': 27,
 'human': 28,
 'organization': 29,
 'system': 30,
 'could': 31,
 'global': 32,
 'economic': 33,
 'much': 34,
 'industry': 35,
 'like': 36,
 'today': 37,
 'effort': 38,
 'result': 39,
 'even': 40,
 'provide': 41,
 'give': 42,
 'market': 43,
 'life': 44,
 'issue': 45,
 'skill': 46,
 'important': 47,
 'peace': 48,
 'next': 49,
 'process': 50,
 'would': 51,
 'national': 52,
 'right': 53,
 'session': 54,
 'help': 55,
 'increase': 56,
 'part': 57,
 'well': 58,
 'power': 59,
 'able': 60,
 'must': 61,
 'use': 62,
 'create': 63,
 'challenge': 64,
 'city': 65,
 'see': 66,
 'find': 67,
 '

In [21]:
w2vmodel = Word2Vec(sentences=bigram_model[docs], vector_size=100, sg=1, hs= 1, seed=33)
bigram_counter = Counter()

for key in w2vmodel.wv.key_to_index.keys():
    if key not in stoplist:
        if len(str(key).split("_")) > 1:
            bigram_counter[key] += w2vmodel.wv.vocab[key].count
            #bigram_counter[key] += w2vmodel.wv.vocab[key].count

for key, counts in bigram_counter.most_common(30):
    print(key,">>>>> " ,counts)

AttributeError: The vocab attribute was removed from KeyedVector in Gensim 4.0.0.
Use KeyedVector's .key_to_index dict, .index_to_key list, and methods .get_vecattr(key, attr) and .set_vecattr(key, attr, new_val) instead.
See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4